# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
#!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [3]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [4]:
from glob import glob

text_lines = []

for file_path in glob("./mfd.md", recursive=True):
    with open(file_path, "r") as file:
        merge_line = ""
        for line in file:
            line = line.strip().replace("------", "")
            if (len(line) == 0):
                continue
            if (line.startswith("#")):
                line = line.replace("#", "")
                text_lines.append(line)
            if (line.startswith("**")):
                if (len(merge_line) > 0 and merge_line not in text_lines):
                    text_lines.append(merge_line)
                merge_line = line.replace("*", "")
            else:
                merge_line += line

    #text_lines += file_text.split("# ")

In [5]:
len(text_lines)
#print(text_lines[3])

415

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [6]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [7]:
#from pymilvus import model as milvus_model

#embedding_model = milvus_model.DefaultEmbeddingFunction()

from sentence_transformers import SentenceTransformer

# 加载 Qwen 嵌入模型
embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B", device="cpu")

生成一个测试嵌入并打印其维度和前几个元素。

In [8]:
test_embedding = embedding_model.encode(["这是一个测试"], prompt_name="query")
embedding_dim = test_embedding.shape[1]
print(embedding_dim)
print(test_embedding[:10])

1024
[[-0.00167848 -0.04044234 -0.00327529 ...  0.01051191 -0.01248816
   0.03865897]]


In [9]:
#test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
#print(test_embedding_0[:10])

## 将数据加载到 Milvus

### 创建 Collection

In [10]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [11]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [12]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [13]:
from tqdm import tqdm

data = []
# text_lines = ["这是一个测试", "中华人民共和国民法典"]
doc_embeddings = embedding_model.encode(text_lines)

#for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
#    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

data = [{"id": i, "vector": embedding, "text": text} for i, (embedding, text) in enumerate(zip(doc_embeddings, text_lines))]

milvus_client.insert(collection_name=collection_name, data=data)


{'insert_count': 415, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [14]:
questions = [
    "矿藏、水流、海域等自然资源属于国家所有吗?",
    "当发生哪些情形时，合同的权利义务应当终止？"
]

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [15]:
search_res_0 = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode(
        [questions[0]], prompt_name="query"
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)
search_res_1 = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode(
        [questions[1]], prompt_name="query"
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [16]:
import json

retrieved_lines_with_distances_0 = [
    (res["entity"]["text"], res["distance"]) for res in search_res_0[0]
]
retrieved_lines_with_distances_1 = [
    (res["entity"]["text"], res["distance"]) for res in search_res_1[0]
]
#print(json.dumps(retrieved_lines_with_distances, indent=4))

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [17]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances_0]
)
context += "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances_1]
)

In [18]:
context

'第二百五十三条 矿藏、水流、海域等自然资源属于国家所有。法律规定属于集体所有的，依照其规定。\n第二百五十一条 矿藏、水流、海域等自然资源属于国家所有。法律规定属于集体所有的，依照其规定。\n第二百六十一条 属于国家所有的自然资源，所有权可以不登记。第五百五十七条 有下列情形之一的，合同的权利义务终止：（一）债务已经履行；（二）合同解除；（三）债务相互抵销；（四）债务人依法将标的物提存；（五）债权人免除债务；（六）债权债务同归一人；（七）法律规定或者当事人约定终止的其他情形。\n 第四章 合同的权利义务终止\n第五百六十七条 合同的权利义务终止后，当事人应当履行清理义务。合同终止后，当事人可以约定合同终止后的权利义务。法律、行政法规对合同终止后的权利义务有特别规定的，依照其规定。'

In [19]:
questions

['矿藏、水流、海域等自然资源属于国家所有吗?', '当发生哪些情形时，合同的权利义务应当终止？']

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [20]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context}
</context>
<question>
{questions[0]}
</question>
<question>
{questions[1]}
</question>
<translated>
</translated>
"""

In [21]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。\n<context>\n第二百五十三条 矿藏、水流、海域等自然资源属于国家所有。法律规定属于集体所有的，依照其规定。\n第二百五十一条 矿藏、水流、海域等自然资源属于国家所有。法律规定属于集体所有的，依照其规定。\n第二百六十一条 属于国家所有的自然资源，所有权可以不登记。第五百五十七条 有下列情形之一的，合同的权利义务终止：（一）债务已经履行；（二）合同解除；（三）债务相互抵销；（四）债务人依法将标的物提存；（五）债权人免除债务；（六）债权债务同归一人；（七）法律规定或者当事人约定终止的其他情形。\n 第四章 合同的权利义务终止\n第五百六十七条 合同的权利义务终止后，当事人应当履行清理义务。合同终止后，当事人可以约定合同终止后的权利义务。法律、行政法规对合同终止后的权利义务有特别规定的，依照其规定。\n</context>\n<question>\n矿藏、水流、海域等自然资源属于国家所有吗?\n</question>\n<question>\n当发生哪些情形时，合同的权利义务应当终止？\n</question>\n<translated>\n</translated>\n'

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [22]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文信息，我将分别回答两个问题：

1. 关于矿藏、水流、海域等自然资源的所有权问题：
根据第二百五十三条和第二百五十一条的规定，矿藏、水流、海域等自然资源属于国家所有。法律规定属于集体所有的，依照其规定。

2. 关于合同权利义务终止的情形：
根据第五百五十七条的规定，有下列情形之一的，合同的权利义务终止：
（一）债务已经履行；
（二）合同解除；
（三）债务相互抵销；
（四）债务人依法将标的物提存；
（五）债权人免除债务；
（六）债权债务同归一人；
（七）法律规定或者当事人约定终止的其他情形。
